# TitaniumCloud networking API-s
This notebook contains code examples for the networking API-s.
**NOTE:** If pasted into a Python file in the displayed order, all code cells can also work as a Python script.

### Covered API classes
This notebook covers examples for the following API classes:
- **AnalyzeURL** (*TCA-0404 - Analyze URL*)
- **URLThreatIntelligence** (*TCA-0403 - URL Threat Intelligence*)
- **DomainThreatIntelligence** (*TCA-0405 - Domain Threat Intelligence*)
- **IPThreatIntelligence** (*TCA-0406 - IP Threat Intelligence*)

### Credentials
Credentials are loaded from a local file instead of being written here in plain text.
To learn how to creat the credentials file, see the **Storing and using the credentials** section in the [README file](./README.md)

### 1. Importing the required classes
First, we will import the required API classes from the ticloud module.

In [ ]:
from ReversingLabs.SDK.ticloud import AnalyzeURL, URLThreatIntelligence, DomainThreatIntelligence, IPThreatIntelligence

### 2. Loading the credentials
Next, we will load our TitaniumCloud credentials from the local `ticloud_credentials.json` file.
**NOTE: Instead of doing this step, you can paste your credentials while creating the Python object in the following step.**

In [ ]:
import json


CREDENTIALS = json.load(open("ticloud_credentials.json"))
USERNAME = CREDENTIALS.get("username")
PASSWORD = CREDENTIALS.get("password")

### 3. Domain threat intelligence

#### Domain report
We will start by fetching threat intelligence data for a domain string. Let us say that the following domain has popped up in a recent analysis campaign someone performed: *"kosmikband.com"*

To get a domain analysis report from a DomainThreatIntelligence object, we need to do the following:

In [ ]:
DOMAIN = "kosmikband.com"

domain_ti = DomainThreatIntelligence(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent="ReversingLabs SDK cookbook"
)

domain_report = domain_ti.get_domain_report(domain=DOMAIN)

report_text = domain_report.text
print(report_text)

From the domain analysis report we can see that several reputation sources classified that domain as malicious. Also, we can see the list of top threats found withing that domain:
```
"top_threats": [
            {
                "threat_name": "ByteCode-MSIL.Trojan.Generic",
                "threat_level": 5,
                "files_count": 2
            },
            {
                "threat_name": "ByteCode-MSIL.Downloader.RemcosRAT",
                "threat_level": 3,
                "files_count": 2
            },
            {
                "threat_name": "ByteCode-MSIL.Trojan.Seraph",
                "threat_level": 5,
                "files_count": 1
            },
            {
                "threat_name": "Win32.Trojan.Generic",
                "threat_level": 5,
                "files_count": 1
            },
            {
                "threat_name": "ByteCode-MSIL.Trojan.RealProtect",
                "threat_level": 5,
                "files_count": 1
            }
        ]
```

#### Domain-to-IP resolutions
Let us take the domain whose report we just analyzed and send it to the Domain-to-IP resolutions endpoint. This will give us a list of IP addresses into which our domain resolves.

In [ ]:
domain_to_ip = domain_ti.domain_to_ip_resolutions(domain=DOMAIN)

domain_to_ip_text = domain_to_ip.text
print(domain_to_ip_text)

In this particular case, we can see that our domain resolves into one IP address and that the provider of this resolution information is ReversingLabs.  
Now we will parse the resolutions report and get that IP address from it.

In [ ]:
resolved_ips = domain_to_ip.json()
resolutions = resolved_ips.get("rl").get("resolutions")
IP_ADDR = resolutions[0].get("ip")

Since we saw that we have only one IP address in that list, it is safe for us to simply fetch it using its position number.  
Now we will store that IP address into the `ip_addr` variable for later use.


### 4. IP threat intelligence
To get a list of URL-s associated with an IP address, we can use the `urls_from_ip` method from the `IPThreatIntelligence` class.  
The IP address is already stored in the `ip_addr` variable and can be used in our next step.

In [ ]:
ip_ti = IPThreatIntelligence(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent="ReversingLabs SDK cookbook"
)

ip_urls = ip_ti.urls_from_ip(ip_address=IP_ADDR)

ip_urls_text = ip_urls.text
print(ip_urls_text)

The list of URL-s associated with our IP address is lengthy. If we want to gather all the valuable information about these URL-s from TitaniumCloud, we need to fetch the analysis report for each of them. In order not to bomb our tutorial with thousands of lines of report, we will only fetch the analysis report for the first URL.

### 5. URL threat intelligence
The URL threat intelligence API also offers several different endpoints for URL analysis. We will demonstrate the results from two of them:

#### URL report
Since we know with confidence that our previously generated list of URL-s contains some entries (is not empty), we can safely parse out just the first URL from that list:


In [ ]:
associated_urls = ip_urls.json()
url_list = associated_urls.get("rl").get("urls")
URL = url_list[0].get("url")

After we have parsed out the URL that we need, we will create a `URLThreatIntelligence` object and fetch an analysis report for our URL.

In [ ]:
url_ti = URLThreatIntelligence(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent="ReversingLabs SDK cookbook"
)

url_report = url_ti.get_url_report(url_input=URL)

report_text = url_report.text
print(report_text)

In this report output we can see the following interesting data:
- Third party reputation sources - A list of third party reputation services and their classification of the provided URL.
- Analysis history - A list of timestamped analyses of the provided URL.
- Both the third party reputations and the analysis sections of the report also contain statistics lists which show how many sources returned which classification.
- An overall classification of the provided URL, alongside the reason for that classification.

#### Downloaded files
Now we will fetch the list of files downloaded from our URL. Each listed file can also contain its reputation information.

In [ ]:
dl_files = url_ti.get_downloaded_files(url_input=URL)

dl_files_text = dl_files.text
print(dl_files_text)

Here we can see a list of files found on our URL that were downloaded and analyzed by ReversingLabs. Each listed file also has its own reputation information. Although the files downloaded from a URL end up classified as `KNOWN`, that does not mean that the domain, IP addresses nor URL-s on which they have been found can be treated as safe. There are other mechanisms and rules for classifying those entities besides just analyzing the files found on them.

### 6. Analyze URL
In case there are no threat intelligence results available for a URL, we can always request them.  
To send a URL for detailed analysis, use the `AnalyzeURL` class.

In [ ]:
analyze = AnalyzeURL(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent="ReversingLabs SDK cookbook"
)

analyze_response = analyze.submit_url(url_input=URL)

analyze_response_text = analyze_response.text
print(analyze_response_text)

If the action was successful, we will see a response which confirms us that the URL was **submitted** for analysis.  
Usually a fresh analysis takes some time to finish, depending on the complexity of the provided URL. You can check the completion of the report along the way.

After the URL analysis is done, we can again fetch any of the available info generated from analyzing the provided URL.


### 7. Advanced
In case you want to configure any pre-set additional parameters in any of the demonstrated methods or if you want to use the methods that hande paging automatically, check the [7. Advanced](./file_reputation_and_analysis.ipynb) section in the reputation notebook.